In [1]:
import tensorflow as tf
import tflearn

import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [11]:
#split data into features (images) and classes (labels)
full_df = pd.read_csv('flattend.csv')
labels = full_df.iloc[:, 0]
labels_OH = pd.get_dummies(labels) #onehot encoded 
names = full_df.iloc[:, 1] #string ID
images = full_df.iloc[:, 2:]
images[images>0]=1  #convert back to binary (compression changed that)

#get subsets of data to test around with 
labels_subset = labels_OH[:4000].values
images_subset = images[:4000].values

#split data into chunks (20% each) for a 5-fold cross validation 
(trainX, testX, trainY, testY) =  train_test_split(images_subset, labels_subset)

#reshape back to normal image shape 
trainX = trainX.reshape([-1, 8, 1536, 1])
testX = testX.reshape([-1, 8, 1536, 1])

In [12]:
# Building convolutional network
tf.reset_default_graph()
network = input_data(shape=[None, 8, 1536, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)

network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)

network = local_response_normalization(network)
network = fully_connected(network, 128, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='tanh')
network = dropout(network, 0.8)
network = fully_connected(network, 4, activation='softmax')

network = regression(network, optimizer='adam', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

# Training
model = tflearn.DNN(network, tensorboard_verbose=0)
#model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=100, n_epoch=100)

model.save('my_model.tflearn')

INFO:tensorflow:C:\Users\mohit\Documents\Python\data_science\deep-learning\intro-to-tflearn\my_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [63]:
y_hat = model.predict(trainX)
y_hat_pd = pd.DataFrame(data=y_hat, columns=labels_OH.columns)
testY_pd = pd.DataFrame(data=trainY, columns=labels_OH.columns)

In [64]:
correct = []
wrong = []

for i in range(len(testX)):
    prediction = y_hat_pd.iloc[i].argmax()
    true = testY_pd.iloc[i].argmax()
    if prediction==true: 
        correct.append(i)
    else:
        wrong.append(i)
print (len(correct), len(wrong))

206 794
